## Read files

In [1]:
import pandas as pd

In [5]:
one_sample = '../Cellranger/BT1_diff/'
file_metabo = 'Human-GEM_v1.14_geneIDs.txt'

In [88]:
# read data from metabo file
metabo_genes = pd.read_csv(file_metabo, sep='\t')
metabo_genes = metabo_genes.rename(columns={'geneEnsembl': 'Gene ID', 'geneSymbols': 'Gene name'})
metabo_genes = metabo_genes[['Gene ID', 'Gene name']]

metabo_genes['Gene ID'].size # 2920

In [89]:
# read data from sample features file
all_genes = pd.read_csv(one_sample+'features.tsv',sep='\t')
all_genes.columns = ["Gene ID","Gene name", "expression"]
all_genes = all_genes[['Gene ID', 'Gene name']]

all_genes['Gene ID'].size

## Merge dataframes on Gene ID

In [116]:
merged_df = pd.merge(metabo_genes, all_genes, on=['Gene ID'], how='inner')

merged_df['Gene ID'].size

In [114]:
## change columns names
merged_df = merged_df.rename(columns={'Gene ID':'geneEnsembl','Gene name_x': 'Toulouse', 'Gene name_y': 'Gene_name'})
merged_df

,geneEnsembl,Toulouse,Gene_name
0,ENSG00000000419,DPM1,DPM1
1,ENSG00000001036,FUCA2,FUCA2
2,ENSG00000001084,GCLC,GCLC
3,ENSG00000001630,CYP51A1,CYP51A1
4,ENSG00000002549,LAP3,LAP3
...,...,...,...
2910,ENSG00000284844,NaN,TOMT
2911,ENSG00000285043,NaN,ENSG00000285043
2912,ENSG00000285269,NaN,ENSG00000285269
2913,ENSG00000288702,UGT1A3,UGT1A3


## Add missing genes (by hand)

In [94]:
## what is missing in cellranger features compared to the metabo_file
test_all = pd.concat([metabo_genes['Gene ID'].dropna(), merged_df['Gene ID'].dropna()], axis=0)
count = test_all.value_counts()
res = count[count==1]
res

ENSG00000161133    1
ENSG00000263464    1
ENSG00000189366    1
ENSG00000133475    1
ENSG00000112096    1
Name: Gene ID, dtype: int64

In [118]:
## add missing genes
gene1 = {'geneEnsembl': 'ENSG00000161133', 'Toulouse': 'ENSG00000161133', 'Gene_name': 'USP41'}
gene2 = {'geneEnsembl': 'ENSG00000263464', 'Toulouse': 'ENSG00000263464', 'Gene_name': 'ENSG00000263464'}
gene3 = {'geneEnsembl': 'ENSG00000189366', 'Toulouse': 'ENSG00000189366', 'Gene_name': 'ALG1L1P'}
gene4 = {'geneEnsembl': 'ENSG00000133475', 'Toulouse': 'ENSG00000133475', 'Gene_name': 'GGT2P'}
gene5 = {'geneEnsembl': 'ENSG00000112096', 'Toulouse': 'ENSG00000112096', 'Gene_name': 'ENSG00000112096'}

merged_df = merged_df.append(gene1, ignore_index = True)
merged_df = merged_df.append(gene2, ignore_index = True)
merged_df = merged_df.append(gene3, ignore_index = True)
merged_df = merged_df.append(gene4, ignore_index = True)
merged_df = merged_df.append(gene5, ignore_index = True)

merged_df

,geneEnsembl,Toulouse,Gene_name
0,ENSG00000000419,DPM1,DPM1
1,ENSG00000001036,FUCA2,FUCA2
2,ENSG00000001084,GCLC,GCLC
3,ENSG00000001630,CYP51A1,CYP51A1
4,ENSG00000002549,LAP3,LAP3
...,...,...,...
2915,ENSG00000161133,ENSG00000161133,USP41
2916,ENSG00000263464,ENSG00000263464,ENSG00000263464
2917,ENSG00000189366,ENSG00000189366,ALG1L1P
2918,ENSG00000133475,ENSG00000133475,GGT2P


In [122]:
merged_df['Gene_name'].str.replace("_", '-')

0                  DPM1
1                 FUCA2
2                  GCLC
3               CYP51A1
4                  LAP3
             ...       
2915              USP41
2916    ENSG00000263464
2917            ALG1L1P
2918              GGT2P
2919    ENSG00000112096
Name: Gene_name, Length: 2920, dtype: object

## See differences in genes names convention

In [120]:
# filter rows where column Gene_name has different values compared to Toulouse
different_rows = merged_df[(merged_df['Toulouse'] != merged_df['Gene_name'])]
print(different_rows)

          geneEnsembl         Toulouse        Gene_name
440   ENSG00000100101              NaN  ENSG00000100101
526   ENSG00000102125              TAZ         TAFAZZIN
2074  ENSG00000166986          mars-01            MARS1
2388  ENSG00000177628              GBA             GBA1
2423  ENSG00000180011            ZADH2            PTGR3
2498  ENSG00000184154           LRTOMT           LRRC51
2857  ENSG00000247626          mars-02            MARS2
2871  ENSG00000255730              NaN  ENSG00000255730
2884  ENSG00000259916              NaN            AQP7B
2895  ENSG00000272916              NaN  ENSG00000272916
2910  ENSG00000284844              NaN             TOMT
2911  ENSG00000285043              NaN  ENSG00000285043
2912  ENSG00000285269              NaN  ENSG00000285269
2915  ENSG00000161133  ENSG00000161133            USP41
2917  ENSG00000189366  ENSG00000189366          ALG1L1P
2918  ENSG00000133475  ENSG00000133475            GGT2P


### Store the results

In [75]:
merged_df.to_csv('all_metabo.tsv',
                 sep="\t", header=True,
                 index=False)